<a href="https://colab.research.google.com/github/nicoRomeroCuruchet/DynamicProgramming/blob/main/testing_bary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import numpy as np
from PolicyIteration import PolicyIteration
from utils.utils import plot_2D_value_function,\
                        plot_3D_value_function,\
                        test_enviroment

# CartPoleEnv 

### Observation Space

The observation is a `ndarray` with shape `(4,)` with the values corresponding to the following positions and velocities:

| Num | Observation           | Min                 | Max               |
|-----|-----------------------|---------------------|-------------------|
| 0   | Cart Position         | -4.8                | 4.8               |
| 1   | Cart Velocity         | -Inf                | Inf               |
| 2   | Pole Angle            | ~ -0.418 rad (-24°) | ~ 0.418 rad (24°) |
| 3   | Pole Angular Velocity | -Inf                | Inf               |

### Action Space

The action is a `ndarray` with shape `(1,)` which can take values `{0, 1}` indicating the direction
of the fixed force the cart is pushed with.

- 0: Push cart to the left
- 1: Push cart to the right

In [ ]:
# Train cartpole environment:
from classic_control.cartpole import CartPoleEnv

env = CartPoleEnv(sutton_barto_reward=True)
# position thresholds:
x_lim         = 2.4
theta_lim     = 0.418 
# velocity thresholds:
x_dot_lim     = 3.1
theta_dot_lim = 3.1

bins_space = {
    "x_space"         : np.linspace(-x_lim, x_lim, 10,  dtype=np.float32),                     # position space         (0)
    "x_dot_space"     : np.linspace(-x_dot_lim, x_dot_lim, 7,  dtype=np.float32),              # velocity space         (1)
    "theta_space"     : np.linspace(-theta_lim, theta_lim, 10, dtype=np.float32),              # angle space            (2)
    "theta_dot_space" : np.linspace(-theta_dot_lim, theta_dot_lim, 7, dtype=np.float32),       # angular velocity space (3)
}

pi = PolicyIteration(
    env=env, 
    bins_space=bins_space,
    action_space=np.array([0, 1], dtype=np.int32),
    gamma=0.99,
    theta=1e-3
)

pi.run()

In [ ]:
# Test cartpole environment:

with open(env.__class__.__name__ + ".pkl", "rb") as f:
    pi = pickle.load(f)

test_enviroment(CartPoleEnv(sutton_barto_reward=True, render_mode="human"), pi)

## Observation Space

The observation is a `ndarray` with shape `(2,)` where the elements correspond to the following:

| Num | Observation                          | Min   | Max  | Unit         |
|-----|--------------------------------------|-------|------|--------------|
| 0   | position of the car along the x-axis | -1.2  | 0.6  | position (m) |
| 1   | velocity of the car                  | -0.07 | 0.07 | velocity (v) |

## Action Space

There are 3 discrete deterministic actions:

- 0: Accelerate to the left
- 1: Don't accelerate
- 2: Accelerate to the right


In [2]:
from classic_control.continuous_mountain_car import Continuous_MountainCarEnv

env=Continuous_MountainCarEnv()

bins_space = {
    "x_space":     np.linspace(env.min_position, env.max_position, 20,      dtype=np.float32),    # position space    (0)
    "x_dot_space": np.linspace(-abs(env.max_speed), abs(env.max_speed), 20, dtype=np.float32),    # velocity space    (1)
}

pi = PolicyIteration(
    env=env, 
    bins_space=bins_space,
    action_space=np.linspace(-1.0, +1.0,9, dtype=np.float32),
    gamma=0.99,
    theta=1e-3,
)
#pi.run()

2024-09-08 18:22:37.935 | INFO     | PolicyIteration:__init__:120 - Lower bounds: [-1.2  -0.07]
2024-09-08 18:22:37.935 | INFO     | PolicyIteration:__init__:121 - Upper bounds: [0.6  0.07]
2024-09-08 18:22:38.015 | INFO     | PolicyIteration:__init__:127 - Creating Delaunay triangulation...
2024-09-08 18:22:38.019 | INFO     | PolicyIteration:__init__:129 - Delaunay triangulation created.
2024-09-08 18:22:38.101 | INFO     | PolicyIteration:__init__:156 - Policy Iteration was correctly initialized.
2024-09-08 18:22:38.102 | INFO     | PolicyIteration:__init__:157 - The enviroment name is: Continuous_MountainCarEnv
2024-09-08 18:22:38.103 | INFO     | PolicyIteration:__init__:158 - The action space is: [-1.   -0.75 -0.5  -0.25  0.    0.25  0.5   0.75  1.  ]
2024-09-08 18:22:38.103 | INFO     | PolicyIteration:__init__:159 - Number of states: 400
2024-09-08 18:22:38.104 | INFO     | PolicyIteration:__init__:160 - Total states:3600


In [ ]:
# Test mountain car environment:
with open(env.__class__.__name__ + ".pkl", "rb") as f:
    pi: PolicyIteration = pickle.load(f)

test_enviroment(Continuous_MountainCarEnv(render_mode="human"), pi)

In [ ]:
# graph the value function of the mountain car environment:
plot_3D_value_function(pi.value_function)
plot_2D_value_function(pi.value_function)

In [3]:
import cProfile, pstats, io
from pstats import SortKey
pr = cProfile.Profile()
pr.enable()
pi.run()
pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

2024-09-08 18:20:53.463 | INFO     | PolicyIteration:run:462 - Generating transition and reward function table...
2024-09-08 18:20:54.806 | INFO     | PolicyIteration:run:464 - Transition and reward function table generated.
  0%|          | 0/100 [00:00<?, ?it/s]2024-09-08 18:20:54.811 | INFO     | PolicyIteration:run:466 - solving step 0
2024-09-08 18:20:54.812 | INFO     | PolicyIteration:policy_evaluation:384 - Starting policy evaluation
2024-09-08 18:20:55.026 | INFO     | PolicyIteration:policy_evaluation:410 - Max Error: 99.95833587646484                            | Avg Error: 6.203125                            | 0<0.001
2024-09-08 18:20:58.895 | INFO     | PolicyIteration:policy_evaluation:410 - Max Error: 8.1044921875                            | Avg Error: 5.029037952423096                            | 0<0.001
2024-09-08 18:21:02.861 | INFO     | PolicyIteration:policy_evaluation:410 - Max Error: 0.658203125                            | Avg Error: 0.5401424169540405        

         51850623 function calls (51844773 primitive calls) in 47.769 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000   47.901   23.951 c:\Users\nicor\anaconda3\envs\DynamicProgramming\Lib\site-packages\IPython\core\interactiveshell.py:3541(run_code)
      3/2    0.000    0.000   47.901   23.951 {built-in method builtins.exec}
        1    0.001    0.001   47.901   47.901 c:\users\nicor\dynamicprogramming\src\PolicyIteration.py:458(run)
        5    0.655    0.131   45.690    9.138 c:\users\nicor\dynamicprogramming\src\PolicyIteration.py:377(policy_evaluation)
    24840    0.347    0.000   34.198    0.001 c:\users\nicor\dynamicprogramming\src\PolicyIteration.py:345(get_value)
    55172    0.306    0.000   28.899    0.001 c:\Users\nicor\anaconda3\envs\DynamicProgramming\Lib\site-packages\jax\_src\array.py:318(__getitem__)
    55181    0.221    0.000   28.388    0.001 c:\Users\nicor\anaconda3\env

In [3]:
import cProfile, pstats, io
from pstats import SortKey
pr = cProfile.Profile()
pr.enable()
pi.run()
pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

2024-09-08 18:22:41.817 | INFO     | PolicyIteration:run:462 - Generating transition and reward function table...
2024-09-08 18:22:42.958 | INFO     | PolicyIteration:run:464 - Transition and reward function table generated.
  0%|          | 0/100 [00:00<?, ?it/s]2024-09-08 18:22:42.964 | INFO     | PolicyIteration:run:466 - solving step 0
2024-09-08 18:22:42.965 | INFO     | PolicyIteration:policy_evaluation:384 - Starting policy evaluation
2024-09-08 18:22:43.160 | INFO     | PolicyIteration:policy_evaluation:410 - Max Error: 99.95833587646484                            | Avg Error: 6.203125                            | 0<0.001
2024-09-08 18:22:45.875 | INFO     | PolicyIteration:policy_evaluation:410 - Max Error: 8.1044921875                            | Avg Error: 5.029037952423096                            | 0<0.001
2024-09-08 18:22:48.544 | INFO     | PolicyIteration:policy_evaluation:410 - Max Error: 0.658203125                            | Avg Error: 0.5401424169540405        

         37094980 function calls (37089144 primitive calls) in 36.303 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000   36.377   18.189 c:\Users\nicor\anaconda3\envs\DynamicProgramming\Lib\site-packages\IPython\core\interactiveshell.py:3541(run_code)
      3/2    0.000    0.000   36.377   18.189 {built-in method builtins.exec}
        1    0.001    0.001   36.377   36.377 c:\users\nicor\dynamicprogramming\src\PolicyIteration.py:458(run)
        5    0.635    0.127   34.423    6.885 c:\users\nicor\dynamicprogramming\src\PolicyIteration.py:377(policy_evaluation)
    55172    0.298    0.000   28.988    0.001 c:\Users\nicor\anaconda3\envs\DynamicProgramming\Lib\site-packages\jax\_src\array.py:318(__getitem__)
    55181    0.216    0.000   28.485    0.001 c:\Users\nicor\anaconda3\envs\DynamicProgramming\Lib\site-packages\jax\_src\numpy\lax_numpy.py:7832(_rewriting_take)
    24840    0.609    0.000  